In [2]:
!pip install -q --upgrade google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import userdata
API_KEY = userdata.get('GOOGLE_API_KEY')

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [1]:
!pip install langchain_community

In [7]:
!pip install pypdf

In [8]:
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader('/content/drive/MyDrive/termsofusesVB.pdf')
loaded_pdf_doc = pdfloader.load()

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=['\n\n', '\n', ' '])

In [10]:
chunks = recursive_splitter.split_documents(loaded_pdf_doc)
print(len(chunks))

124


In [6]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 48.2 MB/s eta 0:00

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001',google_api_key=API_KEY)

In [12]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [13]:
vector_index = vector_store.as_retriever(search_kwargs={'k':8})

In [16]:
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
    model,
    retriever = vector_index,
    return_source_documents=True
)

In [ ]:
question = 'what is spam according to the doc?'
response = rag({"query":question})
print(response)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Extract text content from the chunks
chunk_texts = [chunk.page_content for chunk in chunks]

# Train the vectorizer on your text data
tfidf_vectorizer.fit(chunk_texts)

# Save the trained vectorizer to a .pkl file
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

print("Vectorizer saved as vectorizer.pkl")

Vectorizer saved as vectorizer.pkl


This code trains a `TfidfVectorizer` on the text content of your `chunks` and saves it to a file named `vectorizer.pkl`. You can later load this file to transform new text data using the same vectorizer.

In [18]:
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np

# Load the trained vectorizer
with open('vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

# Prepare data (assuming you have labels for your chunks)
# Replace with your actual labels
labels = np.random.randint(0, 2, size=len(chunks)) # Example random labels

# Vectorize the text data
chunk_texts = [chunk.page_content for chunk in chunks]
vectorized_data = loaded_vectorizer.transform(chunk_texts)

# Choose and train a model (using Logistic Regression as an example)
model = LogisticRegression()
model.fit(vectorized_data, labels)

# Save the trained model to a .pkl file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved as model.pkl")

Model saved as model.pkl
